# How to Save Interactive Bokeh Plots

This guide shows you how to export hvPlot visualizations using the Bokeh backend to various file formats, with a focus on interactive HTML exports and static PNG images.

## Setup

First, let's set up hvPlot with the default Bokeh backend and create a sample plot:

In [ ]:
import hvplot.pandas # noqa

penguins = hvplot.sampledata.penguins("pandas").dropna()

plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    color='species',
    title="Penguin Bill Dimensions"
)

plot

## HTML Export (Recommended)

The primary strength of Bokeh plots is their interactivity. HTML export preserves all interactive features like zooming, panning, and hover tooltips:

In [ ]:
hvplot.save(plot, 'penguins_bokeh.html')
print("Interactive HTML plot saved as 'penguins_bokeh.html'")

## HTML with Inline Resources

By default, HTML files depend on loading JavaScript from online CDN repositories. For offline or airgapped environments, use inline resources:

In [ ]:
hvplot.save(plot, 'penguins_offline.html', resources='inline')
print("Self-contained HTML plot saved as 'penguins_offline.html'")

## PNG Export

For static images, Bokeh can export to PNG, but this requires additional browser driver dependencies.

### Installing PNG Export Dependencies

Choose one of these options:

**Option 1: Chrome/Chromium (Recommended)**

Install Chrome or Chromium browser first, then install the Python package.

```bash
pip install selenium chromedriver-autoinstaller
```

**Option 2: Firefox**

Install Firefox browser first, then install geckodriver.
```bash
pip install selenium
```
Download geckodriver from the official [GitHub releases page](https://github.com/mozilla/geckodriver), then add geckodriver to your PATH

**Option 3: Using conda**
```bash
conda install selenium
```


:::{tip}
For help adding geckodriver to your PATH:
- **Windows:**
  - Place `geckodriver.exe` in a folder (e.g., `C:\\tools\\geckodriver`)
  - Add that folder to your System PATH:
    - _System Properties_ > _Environment Variables_.
    - Edit the Path variable and add the folder path.
- **macOS/Linux:**
  - Move the `geckodriver` binary to `/usr/local/bin` or another directory already in your PATH.
  - Alternatively, update your PATH manually in your shell config (e.g., `export PATH="$PATH:/path/to/geckodriver"`)

If unsure, consult your operating system documentation for instructions on modifying the PATH environment variable.
:::

### Saving PNG Files

Once dependencies are installed:

In [ ]:
try:
    hvplot.save(plot, 'penguins_bokeh.png')
    print("PNG saved successfully as 'penguins_bokeh.png'")
except Exception as e:
    print(f"PNG export failed: {e}")
    print("Make sure you have selenium and a browser driver installed.")

:::{note}
This will only work if you have selenium and a browser driver installed
:::

### High-Resolution PNG Export

For presentations or print, you may want higher resolution images:

In [ ]:
high_res_plot = penguins.hvplot.scatter(
    x='bill_length_mm',
    y='bill_depth_mm',
    width=1200,
    height=800,
    color='species',
    title="High-Resolution Penguin Plot"
)

high_res_plot

In [ ]:
try:
    hvplot.save(high_res_plot, 'penguins_high_res.png')
    print("High-resolution PNG saved")
except Exception as e:
    print(f"PNG export failed: {e}")

## Batch Saving Multiple Plots

You can efficiently save multiple interactive plots:

In [ ]:
plots = {
    'scatter': penguins.hvplot.scatter(x='bill_length_mm', y='bill_depth_mm', color='species', width=400),
    'histogram': penguins.hvplot.hist('body_mass_g', by='species', alpha=0.7, width=400),
    'box': penguins.hvplot.box(y='flipper_length_mm', by='species', width=400),
}

# Display the plots
(plots['scatter'] + plots['histogram'] + plots['box']).cols(2)

In [ ]:
for name, plot in plots.items():
    # Save interactive HTML
    hvplot.save(plot, f'{name}_bokeh.html')
    print(f"Saved {name}_bokeh.html")

    # Try PNG export
    try:
        hvplot.save(plot, f'{name}_bokeh.png')
        print(f"Saved {name}_bokeh.png")
    except Exception as e:
        print(f"PNG export failed for {name}: {e}")

## Testing PNG Export Setup

Here's how to test if your PNG export setup is working:

In [ ]:
# Test if selenium can find your browser
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    # Test Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    driver = webdriver.Chrome(options=chrome_options)
    driver.quit()
    print("✓ Chrome driver is working")

except Exception as e:
    print(f"✗ Chrome driver issue: {e}")

    # Try Firefox as fallback
    try:
        from selenium.webdriver.firefox.options import Options as FirefoxOptions
        firefox_options = FirefoxOptions()
        firefox_options.add_argument("--headless")
        driver = webdriver.Firefox(options=firefox_options)
        driver.quit()
        print("✓ Firefox driver is working")
    except Exception as e2:
        print(f"✗ Firefox driver issue: {e2}")
        print("Consider installing browser drivers or using matplotlib backend for PNG export")

## Working with Large Datasets

For very large datasets, consider using datashader for efficient rendering:

In [ ]:
large_data = hvplot.sampledata.synthetic_clusters("pandas")
print(f"Dataset size: {len(large_data):,} points")

In [ ]:
datashaded_plot = large_data.hvplot.scatter(
    x='x',
    y='y',
    by='cat',
    datashade=True,
    width=400,
    height=400,
    title="Large Dataset with Datashader"
)

datashaded_plot

In [ ]:
hvplot.save(datashaded_plot, 'large_dataset_bokeh.html')
print("Large dataset plot saved successfully")

## Cleanup

Let's clean up the files we created during this demonstration:

In [ ]:
from pathlib import Path

files_to_clean = [
    'penguins_bokeh.html', 'penguins_offline.html', 'penguins_bokeh.png',
    'penguins_high_res.png',
    'scatter_bokeh.html', 'histogram_bokeh.html', 'box_bokeh.html',
    'scatter_bokeh.png', 'histogram_bokeh.png', 'box_bokeh.png',
    'large_dataset_bokeh.html'
]

for filename in files_to_clean:
    file_path = Path(filename)
    if file_path.exists():
        file_path.unlink()
        print(f"Removed {filename}")
    else:
        print(f"{filename} not found")

## Best Practices for Bokeh Plots

1. **Prioritize HTML export**: Bokeh's strength is interactivity, so HTML should be your primary export format

2. **Use inline resources for offline sharing**: Include `resources=INLINE` when sharing files that won't have internet access

3. **Test PNG export setup**: Always verify your selenium and browser driver installation

4. **Consider file sizes**: Interactive HTML files can be large for complex plots, but preserve all functionality

5. **Handle large datasets**: Use `datashade=True` or `rasterize=True` for datasets with many points

6. **Optimize for your use case**:
   - **Web embedding**: HTML with CDN resources (default)
   - **Offline sharing**: HTML with inline resources
   - **Presentations/documents**: PNG (requires browser driver)
   - **Email attachments**: PNG (smaller, universally viewable)

:::{seealso}
- Learn about [saving matplotlib plots](save_matplotlib_plots.ipynb) for publication-quality static images
- Explore [using Panel for advanced layouts and interactivity](use_panel_for_display.ipynb)
:::